# Classifying which Veterans enroll in the VA
---

This is the biggest question.  If we can identify traits based on which veterans enroll and which do not could have impact on getting vets the help that they need.

In [1]:
# import libraries
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, accuracy_score, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVR, LinearSVR, SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from statsmodels.discrete.discrete_model import Logit
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [11]:
# Read in dataset.  In this case the 5yr veteran dataset is being used
vet = pd.read_csv("../veteran_population_-analysis/working_data/vets_5yr_clean.csv")

'''Additional viewing options'''
# This code sets the notebook to display maximum columns.  Uncomment it to see trunacted view
pd.set_option('display.max_columns', len(vet))  

# This resets the view to a truncated output
#pd.reset_option('display.max_rows')

vet.head()

FileNotFoundError: [Errno 2] File ../vets_5yr_clean.csv does not exist: '../vets_5yr_clean.csv'

# Feature Groups
---
The PUMs dataset is incredibly feature rich.  However, this also makes makes it unwieldy when the desire is only test a specific group of features.  For the sake of simplication, I grouped all of the features into groups, to make for simpler modeling.  Additionally, there are 2 versions of the groups:

* **Non-Encoded** - This uses all categorical features in their orginal state.
* **One Hot Encoded** - This uses expanded categorical variables.

#### Group Definitions

* **Veteran Specific Features** = ```veteran``` 
* **Health and Well Being Features** = ```health```
* **Employment and Income Features** = ```work_income```
* **Ethnicity Features** = ```ethnicity```
* **Lifestyle Features** = ```lifestyle```
* **Geographic Location Features** = ```location```
* **General Demographic Features** = ```general```
* **Education Features** = ```education```




In [ ]:
'''The feature names in this data set are genreally hard to read.  
This code block puts them into lists of corresponding variables.'''

# Non-encoded variables
veteran = ['DRAT', 'DRATX', 'HINS6', 'MIL', 'MLPA', 'MLPB', 'MLPCD', 'MLPE', 'MLPFG', 'MLPH', 'MLPI', 'MLPJ', 'MLPK',
          'VPS', 'FHINS5C', 'FHINS5P', 'FHINS6P', 'FMILSP']
health = ['DDRS', 'DEAR', 'DEYE', 'DOUT', 'DPHY', 'DRAT', 'DREM', 'HINS1', 'HINS2', 'HINS3', 'HINS4', 'HINS5', 'HINS6', 
          'MIG', 'DIS', 'HICOV']
work_income = ['COW', 'INTP_adj', 'OIP_adj', 'NWAB', 'NWLK', 'PAP_adj', 'RETP_adj', 'SEMP_adj', 'SSIP_adj', 'SSP_adj', 
              'WAGP_adj', 'WKHP', 'WKL', 'WKW', 'WRK', 'PINCP', 'PERNP', 'POVPIP', 'ESR']
ethnicity = ['CIT', 'ENG', 'HISP', 'RAC1P', 'RACAIAN', 'RACASN', 'RACBLK', 'RACNH', 'RACNUM', 'RACPI', 'RACSOR', 'RACWHT', 
            'FCITP', 'LANX']
lifestyle = ['JWRIP_1', 'JWRIP_2', 'MAR_1',	'MAR_2', 'MAR_3', 'MAR_4', 'MARHD', 'MARHT', 'MARHW', 'MSP']
location = ['DIVISION', 'REGION', 'ST', 'PUMA', 'intercept']
general = ['AGEP', 'MAR', 'SEX']
education = ['SCH', 'SCHL', 'SCIENGP', 'SCIENGRLP']

# One Hot Encoded variables
veteran_e = ['DRAT', 'DRATX_2', 'HINS6_2', 'MIL_2', 'MIL_3', 'MLPA_0', 'MLPA_1', 'MLPB_0', 'MLPB_1', 'MLPCD_0', 'MLPCD_1', 'MLPE_0',
             'MLPE_1', 'MLPFG_0', 'MLPFG_1', 'MLPH_0', 'MLPH_1', 'MLPI_0', 'MLPI_1', 'MLPJ_0', 'MLPJ_1', 'MLPK_0', 'MLPK_1',
             'VPS_1', 'VPS_2', 'VPS_3', 'VPS_4', 'VPS_5', 'VPS_6', 'VPS_7', 'VPS_8', 'VPS_9', 'VPS_10', 'VPS_11', 'VPS_12', 
             'VPS_13', 'VPS_14', 'VPS_15', 'ESR_1',	'ESR_2', 'ESR_3', 'ESR_4', 'ESR_5', 'FHINS5C', 'FHINS5P', 'FHINS6P', 
             'FMILSP']
health_e = ['DDRS', 'DEAR_2', 'DEYE', 'DOUT_2', 'DPHY_2', 'DRAT', 'DREM_2', 'HINS1_2', 'HINS2_2', 'HINS3_2', 'HINS4_2', 
            'HINS5_2', 'HINS6_2', 'DIS', 'HICOV_2']
work_income_e = ['COW_1', 'COW_2', 'COW_3', 'COW_4', 'COW_5', 'COW_6', 'COW_7', 'COW_8', 'COW_9', 'INTP_adj', 'OIP_adj', 
                  'NWLK_1', 'NWLK_2', 'PAP_adj', 'RETP_adj', 'SEMP_adj', 'SSIP_adj', 'SSP_adj', 
                 'WAGP_adj', 'WKHP', 'WKL', 'WKW_1', 'WKW_2', 'WKW_3', 'WKW_4', 'WKW_5', 'WKW_6', 'WRK_1', 'WRK_2', 
                 'PINCP_adj', 'PERNP_adj', 'POVPIP', 'ESR_1', 'ESR_2', 'ESR_3', 'ESR_4', 'ESR_5']
ethnicity_e = ['CIT_2', 'CIT_3', 'CIT_4', 'CIT_5', 'ENG_1', 'ENG_2', 'ENG_3', 'ENG_4', 'HISP', 'RAC1P', 'RACAIAN', 'RACASN', 
               'RACBLK', 'RACNH', 'RACNUM', 'RACSOR', 'RACWHT', 'FCITP', 'LANX']
lifestyle_e = ['JWRIP', 'MAR_1', 'MAR_2', 'MAR_3', 'MAR_4', 'MARHD_1', 'MARHD_2', 'MARHT_1', 'MARHT_2', 'MARHT_3', 
               'MARHW_1', 'MARHW_2', 'MSP_1', 'MSP_2', 'MSP_3', 'MSP_4', 'MSP_5']
location_e = ['DIVISION_2', 'DIVISION_3', 'DIVISION_4', 'DIVISION_5', 'DIVISION_6', 'DIVISION_7', 'DIVISION_8', 
              'DIVISION_9', 'REGION_2', 'REGION_3', 'REGION_4', 'ST', 'PUMA']
general_e = ['AGEP', 'MAR_1', 'MAR_2', 'MAR_3', 'MAR_4', 'SEX_2']
education_e = [#'SCH_2', 'SCH_3', 'SCH_2', 'SCH_3', 'SCHL_2', 'SCHL_3', 'SCHL_4', 'SCHL_5', 'SCHL_6', 'SCHL_7', 
               'SCHL_8', 'SCHL_9', 'SCHL_10', 'SCHL_11', 'SCHL_12', 'SCHL_13', 'SCHL_14', 'SCHL_15', 'SCHL_16', 
               'SCHL_17', 'SCHL_18', 'SCHL_19', 'SCHL_20', 'SCHL_21', 'SCHL_22', 'SCHL_23', 'SCHL_24', 'SCIENGP', 
               'SCIENGRLP']



In [ ]:
vet['HINS6_2'].value_counts()

## Functions for running various classifier models and scoring the results
---
There are a lot of vairables to test and this is will simplify the process

In [ ]:
# Create function that takes a set of dependent and independent features and splits and scales them

def split_scaler(X, y):

    #Train, test, split
    indep_train, indep_test, dep_train, dep_test = train_test_split(X,
                                                    y,
                                                    stratify=y,  # both veterans and non VA enrollees are highly unbalanced
                                                    test_size=0.3,
                                                    random_state=42)
    # Scale data
    sc = StandardScaler()

    # Fit and transform, while keping column labels
    indep_train_sc = pd.DataFrame(sc.fit_transform(indep_train), columns=indep_train.columns, index=indep_train.index)
    indep_test_sc = pd.DataFrame(sc.transform(indep_test), columns=indep_test.columns, index=indep_test.index)
    
    # Instantiate PCA reduce the dimensionality of the features
    pca = PCA(random_state = 69) 
    
    # Fit PCA on the training data, while keping column labels
    Z_train = pd.DataFrame(pca.fit(indep_train_sc), columns=indep_train.columns, index=indep_train.index)
    Z_test = pd.DataFrame(pca.transform(indep_test), columns=indep_test.columns, index=indep_test.index)
    
    return dep_train, dep_test, indep_train_sc, indep_test_sc, Z_train, Z_test

In [ ]:
def classifier(model, dep_train, dep_test, indep_train, indep_test):
    
    #Instantiate
    model_i = model
    
    #Fit the model
    model_fit = model_i.fit(indep_train, dep_train)
    intercept = model_fit.intercept_
    # Cross Validate
    cv = cross_val_score(model_i, indep_train, dep_train, cv=7).mean()
    
    # Score the model
    print(f" Training Score :                      {model_fit.score(indep_train, dep_train)}")
    print(f" Test Score :                          {model_fit.score(indep_test, dep_test)}")
    print(f" Cross Valditation Score:              {cv}")
    print(f" Intercept:                            {intercept}")
    
    # Generate predictions.
    preds = model_fit.predict(indep_test)
    
    
    # Confusion Matrix
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)

    plot_confusion_matrix(model_fit, 
                          indep_train, 
                          dep_train, 
                          cmap="Purples", 
                          values_format='4g',
                          ax=ax1);
    ax1.set_title('Train Confusion Matrix', fontsize=14);

    plot_confusion_matrix(model_fit, 
                          indep_test, 
                          dep_test, 
                          cmap="Oranges", 
                          values_format='4g',
                          ax=ax2);
    ax2.set_title('Test Confusion Matrix', fontsize=14);
    plt.tight_layout()
    
    # Match coefficients to variables
    var_coef = pd.DataFrame(list(zip(indep_train, model_fit.coef_)), columns=['Indy Variable', 'Coefficient'])
    
    #Classification report on test
    print('')
    print('')
    print('CLASSIFICATION REPORT')
    print(classification_report(dep_test, preds))
    print('')
    #print([pd.DataFrame(var_coef)])
    
    return var_coef

In [ ]:
def log_prob(model):
    
    # Display Odds and Probablities of results
    results = pd.DataFrame(model.params)
        
    results['Odds'] = np.exp(model.params)
    results['Probability'] = results['Odds']/(1+results['Odds'])
    
    # Rename columns
    results = results.reset_index()   
    results.columns = {'index':'Name', 0:'Logit', 'Odds':'Odds', 'Probability':'Probability'}
    print('Odds veteran will NOT enroll: \n', '\n', )
    
    return results

## Modeling Data
---
During exploratory research, several test sets of features were modeled using a variety of classifier models. 

1. Logisitic Regression
2. K Nearest Neighbors
3. Random Forest
4. Gaussian Naive Bayes
5. Bernoulli Naive Bayes
6. Multinomial Naive Bayes
7. Decision Trees
8. Random Forest
9. Extra Trees
10. ADA Boost
11. Guassian Process
12. Support Vector Classifier
13. Linear Support Vector Classifier

In the end, Logistic Regression consistently out performed the other models. It demonstrated to fit better, minimize variance, take less computing power and deliver interpretable results.  Therefore, the preferred classifier going forward is Logistic Regression.

Additionally, as part of model improvement, any features that did not prove to be statistically significant were removed from successive models.  These removed variables are recorded in the ```.drop``` portion of the ```features``` variable for each model.

## Classify VA enrollment based on Veteran Specific Features
---

In [ ]:
# Building modeling criteria
features = vet[vet[veteran_e].drop(['HINS6_2', 'MLPA_0', 'MLPA_1', 'MLPB_1', 'MLPB_0','VPS_1', 'VPS_2', 
                                        'VPS_3', 'VPS_4', 'VPS_5', 'VPS_6', 'VPS_7', 'VPS_8', 'VPS_9', 'VPS_10', 
                                        'VPS_11', 'VPS_12', 'VPS_13', 'VPS_14', 'VPS_15', 'MLPE_0', 'MLPE_1',
                                        'MLPFG_0', 'MLPFG_1', 'MLPH_0', 'MLPH_1', 'MLPI_0', 'MLPI_1', 'MLPJ_0', 
                                        'MLPK_0', 'ESR_5', 'MLPJ_1', 'MLPK_1', 'ESR_2', 'ESR_3'], axis=1).columns]

# Split and Scale
y_train, y_test, X_train_sc, X_test_sc, Z_train, Z_test = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_vet = classifier(LogisticRegression(n_jobs=-1), y_train, y_test, X_train_sc, X_test_sc)

In [ ]:
# Creating a constant for intercept analysis
exog = sm.add_constant(X_train_sc)

# Checking coeffiecients with Stats Models
model = sm.Logit(y_train, exog).fit()
model.summary()

In [ ]:
log_prob(model)

## Classify VA enrollment based on Health and Wellness Features
---


In [ ]:
# Building modeling criteria
features = vet[vet[health_e].drop(['HINS6_2', 'DEYE'], axis=1).columns]

# Split and Scale
y_train1, y_test1, X_train_sc1, X_test_sc1, Z_train1, Z_test1 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_health = classifier(LogisticRegression(n_jobs=-1), y_train1, y_test1, X_train_sc1, X_test_sc1)

In [ ]:
# Creating a constant for intercept analysis
exog1 = sm.add_constant(X_train_sc1)

# Checking coeffiecients with Stats Models
model1 = sm.Logit(y_train1, exog1, missing='drop').fit()
model1.summary()

In [ ]:
log_prob(model1)

## Classify VA enrollment based on Work and Income Features
---


In [ ]:
# Building modeling criteria
features = vet[vet[work_income_e].drop(['COW_1', 'COW_2', 'COW_3', 'COW_4', 'COW_5', 'COW_6', 'COW_7', 
                                            'COW_8', 'COW_9', 'WKW_1', 'WKW_2', 'WKW_3', 'WKW_4', 'WKW_5', 
                                            'WKW_6', 'WKL', 'PERNP_adj', 'PAP_adj', 'RETP_adj', 'SEMP_adj',
                                            'SSIP_adj', 'SSP_adj', 'WAGP_adj'], axis=1).columns]

# Split and Scale
y_train2, y_test2, X_train_sc2, X_test_sc2, Z_train2, Z_test2 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_work = classifier(LogisticRegression(n_jobs=-1), y_train2, y_test2, X_train_sc2, X_test_sc2)

In [ ]:
# Creating a constant for intercept analysis
exog2 = sm.add_constant(X_train_sc2)

# Checking coeffiecients with Stats Models
model2 = sm.Logit(y_train2, exog2, missing='drop').fit()
model2.summary()

In [ ]:
log_prob(model2)

## Classify VA enrollment based on Ethnicity Features
---


In [ ]:
# Building modeling criteria
features = vet[vet[ethnicity_e].drop(['CIT_2', 'CIT_3', 'ENG_1', 'ENG_2', 'ENG_3', 'ENG_4', 'HISP', 'RAC1P',
                                         'RACASN', 'RACNH', 'RACNUM', 'RACSOR', 'RACWHT', 'LANX'], 
                                         axis=1).columns]

# Split and Scale
y_train3, y_test3, X_train_sc3, X_test_sc3, Z_train3, Z_test3 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_work = classifier(LogisticRegression(n_jobs=-1), y_train3, y_test3, X_train_sc3, X_test_sc3)

In [ ]:
# Creating a constant for intercept analysis
exog3 = sm.add_constant(X_train_sc3)

# Checking coeffiecients with Stats Models
model3 = sm.Logit(y_train3, exog3, missing='drop').fit()
model3.summary()

In [ ]:
log_prob(model3)

## Classify VA enrollment based on Lifestyle Features
---
None of these features 

In [ ]:
# Building modeling criteria
features = vet[vet[lifestyle_e].drop(['MAR_1', 'MAR_2', 'MAR_4', 'MAR_3', 'MARHD_1', 'MARHD_2', 'MARHT_1',
                                     'MARHT_2', 'MARHT_3', 'MARHW_1', 'MARHW_2', 'MSP_1', 'MSP_2', 'MSP_3',
                                     'MSP_4', 'MSP_5'], axis=1).columns]

# Split and Scale
y_train4, y_test4, X_train_sc4, X_test_sc4, Z_train4, Z_test4 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_work = classifier(LogisticRegression(n_jobs=-1), y_train4, y_test4, X_train_sc4, X_test_sc4)

In [ ]:
# Creating a constant for intercept analysis
exog4 = sm.add_constant(X_train_sc4)

# Checking coeffiecients with Stats Models
model4 = sm.Logit(y_train4, exog4, missing='drop').fit()
model4.summary()

In [ ]:
log_prob(model4)

## Classify VA enrollment based on Location Features
---
Nothing of interest in this group.  Both state(ST) and PUMA are not true categorizations.  We can effectively say that location is not significant.

In [ ]:
# Building modeling criteria
features = vet[vet[location_e].drop([], axis=1).columns]

# Split and Scale
y_train5, y_test5, X_train_sc5, X_test_sc5, Z_train5, Z_test5 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_work = classifier(LogisticRegression(n_jobs=-1), y_train5, y_test5, X_train_sc5, X_test_sc5)

In [ ]:
# Creating a constant for intercept analysis
exog5 = sm.add_constant(X_train_sc5)

# Checking coeffiecients with Stats Models
model5 = sm.Logit(y_train5, exog5, missing='drop').fit()
model5.summary()

In [ ]:
log_prob(model5)

## Classify VA enrollment based on General Demographic Features
---


In [ ]:
# Building modeling criteria
features = vet[vet[general_e].drop(['MAR_2'], axis=1).columns]

# Split and Scale
y_train6, y_test6, X_train_sc6, X_test_sc6, Z_train6, Z_test6 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_work = classifier(LogisticRegression(n_jobs=-1), y_train6, y_test6, X_train_sc6, X_test_sc6)

In [ ]:
# Creating a constant for intercept analysis
exog6 = sm.add_constant(X_train_sc6)

# Checking coeffiecients with Stats Models
model6 = sm.Logit(y_train6, exog6, missing='drop').fit()
model.summary()

In [ ]:
log_prob(model6)

## Classify VA enrollment based on Education Features
---


In [ ]:
# Building modeling criteria
features = vet[vet[education_e].drop(['SCHL_8', 'SCHL_9', 'SCHL_10', 'SCHL_14', 'SCHL_16'], axis=1).columns]

# Split and Scale
y_train7, y_test7, X_train_sc7, X_test_sc7, Z_train7, Z_test7 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_work = classifier(LogisticRegression(n_jobs=-1), y_train7, y_test7, X_train_sc7, X_test_sc7)

In [ ]:
# Creating a constant for intercept analysis
exog7 = sm.add_constant(X_train_sc7)

# Checking coeffiecients with Stats Models
model7 = sm.Logit(y_train7, exog7, missing='drop').fit()
model7.summary()

In [ ]:
log_prob(model7)

## Final Combined Model
---

In [ ]:
# Building modeling criteria for the top performing features

top_features = ['DRAT', 'DRATX', 'MIL', 'MLPCD_0', 'MLPCD_1', 'ESR_1', 'ESR_4',   #VETERAN FEATURES
                'FHINS5C', 'FHINS5P', 'FMILSP', 
                'DDRS', 'DEAR_2', 'DEYE', 'DOUT_2', 'DPHY_2', 'DREM_2', 'HINS1_2',#HEALTH FEATURES
                'HINS2_2', 'HINS3_2', 'HINS4_2', 'HINS5_2', 'DIS', 'HICOV_2', 
                'INTP_adj', 'OIP_adj', 'PINCP_adj', 'NWLK_1', 'NWLK_2', 'WKHP',   #WORK/INCOME FEATURES
                'WRK_1', 'WRK_2', 'POVPIP', 'ESR_2', 'ESR_3', 'ESR_5',
                'CIT_4', 'CIT_5', 'RACBLK', 'FCITP', 'RACAIAN',                   #ETHNICITY FEATURES
                'AGEP', 'MAR_1', 'MAR_4', 'SEX_2',                                #GENERAL FEATURES
                'SCHL_11', 'SCHL_12', 'SCHL_13', 'SCHL_17', 'SCHL_18', 'SCHL_19', #EDUCATION FEATURES
                'SCHL_20', 'SCHL_21', 'SCHL_22', 'SCHL_23', 'SCHL_24', 'SCIENGP']

features = vet[vet[top_features].drop(['ESR_1', 'ESR_4','FHINS5C', 'DEYE', 'HINS5_2', 'WKHP', 'CIT_5',
                                      'MAR_4', 'SCHL_11', 'SCHL_12', 'SCHL_13', 'INTP_adj'], axis=1).columns]

# Split and Scale
y_train8, y_test8, X_train_sc8, X_test_sc8, Z_train8, Z_test8 = split_scaler(features, vet['HINS6_2'])

# Logitistice Regression (scaled)
var_coef_work = classifier(LogisticRegression(n_jobs=-1), y_train8, y_test8, X_train_sc8, X_test_sc8)

In [ ]:
# Creating a constant for intercept analysis
exog8 = sm.add_constant(X_train_sc8)

# Checking coeffiecients with Stats Models
model8 = sm.Logit(y_train8, exog8, missing='drop').fit()
model8.summary()

In [ ]:
results = log_prob(model8)
results

In [ ]:

#probs = results.drop(labels=['Odds']) #, axis=1)
plt.figure(figsize=(12,10))
sns.heatmap(results['index', 'Probability'], annot=True, cmap="rainbow");

In [ ]:
# Looking at top preforming features that indicate a Canon post
results = results.T
probs = results.drop(labels=0, axis=1)
top_features = results[results['index']]


plt.figure(figsize=(4,8))
sns.heatmap(top_features.sort_values(probs['Probability'], ascending=False), annot=True);

In [ ]:
results.dtypes

In [ ]:
results[0]